### Day 21

In [1]:
import numpy as np
from collections import defaultdict, Counter

In [2]:
day_i = 21

In [3]:
%run start_day.py $day_i

Initializing day 21


In [4]:
cd /home/vincent/Documents/AdventOfCode/2021

/home/vincent/Documents/AdventOfCode/2021


In [5]:
PATH = f"day{day_i}/input{day_i}"

In [6]:
!wc -l $PATH

2 day21/input21


In [7]:
!head $PATH

Player 1 starting position: 10
Player 2 starting position: 6


In [8]:
!tail $PATH

Player 1 starting position: 10
Player 2 starting position: 6


In [9]:
inputs = (10, 6)
inputs_1 = (4, 8)

In [121]:
def loop(v):
    if v > 100:
        return v % 100
    else: return v
    
def roll_die(die):
    value = loop(die + 1) + loop(die + 2) + loop(die + 3)
    return value, loop(die + 3)

def solve1(inp):
    die = n_die = 0
    p1, p2 = inp
    s1 = s2 = 0
    while s1 < 1000 and s2 < 1000:
        move1, die = roll_die(die)
        p1 = 10 if (p1 + move1) % 10 == 0 else (p1 + move1) % 10
        s1 += p1
        n_die += 3
        if s1 >= 1000:
            return s2 * n_die
        move2, die = roll_die(die)
        p2 = 10 if (p2 + move2) % 10 == 0 else (p2 + move2) % 10
        s2 += p2
        n_die += 3
    return s1 * n_die

outcomes = {
    3: 1,
    4: 3,
    5: 6,
    6: 7,
    7: 6,
    8: 3,
    9: 1
}

def play_turn(scores):
    turn_to_play = max(t for _, _, t in scores)
    new_scores = scores.copy()
    updated = False
    for (pos, sco, turn), uni in scores.items():
        if turn == turn_to_play and sco < 21:
            updated = True
            for roll, mult in outcomes.items():
                new_pos = 10 if (pos + roll) % 10 == 0 else (pos + roll) % 10
                new_sco = sco + new_pos
                new_scores[(new_pos, new_sco, turn + 1)] += uni * mult
    return new_scores, updated

def solve2(inp):
    """
    returns:
    - number of wins by 1
    - number of wins by 2
    - who wins most
    """
    p1, p2 = inp
    # PLAYING GAMES
    s1 = defaultdict(int)
    s2 = defaultdict(int)
    s1[(p1, 0, 0)] = 1  # position, score, turn
    s2[(p2, 0, 0)] = 1
    # all games in all universes are played for both players independently
    # we keep the intermediate states as well
    # player 1
    s1, updated1 = play_turn(s1)
    while updated1:
        s1, updated1 = play_turn(s1)
    # player 2
    s2, updated2 = play_turn(s2)
    while updated2:
        s2, updated2 = play_turn(s2)
    # COUNTING WINS
    wins1 = wins2 = 0
    wins_by_1_by_turn = {}
    wins_by_2_by_turn = {}
    max_turns = max(t for _, _, t in list(s1.keys()) + list(s2.keys()))
    # for each turn we count the number of universes in which each player wins the game 
    # (independently from the other)
    for t in range(2, max_turns + 1):
        wins_by_1_by_turn[t] = sum(s1[k] for k in s1 if k[2] == t and k[1] >= 21)
        wins_by_2_by_turn[t] = sum(s2[k] for k in s2 if k[2] == t and k[1] >= 21)
    # then we can just find the total number of universes in which each player wins the game at each turn
    # and add them up
    for t in range(2, max_turns + 1):
        if wins_by_1_by_turn[t] > 0:
            wins1 += wins_by_1_by_turn[t] * sum(s2[k] for k in s2 if k[2] == t-1 and k[1] < 21)
        if wins_by_2_by_turn[t] > 0:
            wins2 += wins_by_2_by_turn[t] * sum(s1[k] for k in s1 if k[2] == t and k[1] < 21)
        
    return wins1, wins2, int(wins2 > wins1) + 1
        

In [115]:
solve1(inputs_1)

739785

In [116]:
solve1(inputs)

900099

In [122]:
solve2(inputs_1)

(444356092776315, 341960390180808, 1)

In [123]:
solve2(inputs)

(306719685234774, 145727874122839, 1)